In [7]:
import numpy as np
import pandas as pd

'''This notebook uses a function that calculates thermodynamic relations for a turbojet, generates a potential solution space based on
real-world values, and then uses numpy minimization and data cleaning techniques to find the ideal combination of compressor (stagnation)
pressure ratio (Pr_c), fuel to air ratio for burner (f), and fuel to air ratio for afterburner (f_ab) to produce the minimum TSFC (thrust
specific fuel consumption) while producing the minimum thrust required. For Dr. Vigor Yang's AE 4451 class at Georgia Tech - Fall 2022

Note: Make sure to adjust create output file names in cells 'subsonic' or 'supersonic' based on operating flight conditions.

Authors: Justin Effendi, Chuck Li, Avi Goel
'''

In [70]:
def turbojet_fun(Pr_c, f, f_ab):
    # Turbojet equations for flight condition: low altitude subsonic cruise
    # Requirements:
    # ST > 380 N * s / kg
    # TSFC minimized
    # T_max_burner = 1300 K
    # T_max_afterburner = 2200 K

    # This is a function that calculates thermodynamic values/relations for a turbojet.
    # Adjust T_a, P_a, and M for given flight condition.

    # Given values (flight conditions)
    T_a = 210 #K
    P_a = 10000 #Pa
    M = 2.6

    gamma_a = 1.4
    gamma_2 = 1.4
    gamma_3 = 1.38
    gamma_4 = 1.33
    gamma_5 = 1.33
    gamma_6 = 1.32
    gamma_e = 1.35
    eta_d = 0.92
    eta_b = 0.99
    eta_pc = 0.9 # Polytropic
    eta_pump = 0.35
    eta_pt = 0.92 # Polytropic
    eta_ab = 0.96
    eta_n = 0.95
    Pr_b = 0.98
    Pr_ab = 0.97
    deltahr = 45E6 # J / kg
    rho_f = 780
    P_f1 = 104E3 # Pa
    deltap_inj = 550E3 # Pa
    RR = 8.314 # J / mol * K
    MolMass = .0288 # kg / mol
    R = RR / MolMass

    # Flight speed, station a
    u = M * np.sqrt(gamma_a * R * T_a)

    # Diffuser outlet, station 2
    if (M < 1):
        r_d = 1
    else:
        r_d = 1 - 0.075 * np.power((M - 1), 1.35)
    T_o2 = T_a * (1 + ((gamma_2 - 1) / 2) * np.power(M,  2))
    P_o2 = P_a * r_d * np.power((1 + eta_d * ((gamma_2 - 1) / 2) * np.power(M, 2)), (gamma_2 / (gamma_2 - 1)))

    # Compressor outlet, station 3
    C_p3 = (gamma_3 * R) / (gamma_3 - 1)
    T_o3 = T_o2 * np.power(Pr_c, (R / (C_p3 * eta_pc)))
    P_o3 = Pr_c * P_o2
    w_c = C_p3 * (T_o3 - T_o2)

    # Burner outlet, station 4
    P_o4 = P_o3 * Pr_b
    C_p4 = (gamma_4 * R) / (gamma_4 - 1)
    T_o4 =  (((f * eta_b * deltahr) / (C_p4 * T_o3)) + 1) / ((f + 1) / T_o3)

    # Fuel pump
    P_f2 = deltap_inj + P_o3
    w_p = ((f + f_ab) * (P_f2 - P_f1)) / (eta_pump * rho_f)

    # Turbine outlet, station 5
    C_p5 = (gamma_5 * R) / (gamma_5 - 1)
    T_o5 = T_o4 - (w_c + w_p) / ((1 + f) * C_p5)
    P_o5 = P_o4 * np.power((T_o5 / T_o4), (C_p5 / (R * eta_pt)))

    # Afterburner outlet, station 6
    C_p6 = (gamma_6 * R) / (gamma_6 - 1)
    T_o6 = ((1 + f) * T_o5 + ((eta_ab * f * deltahr) / C_p6)) / (1 + f + f_ab)
    P_o6 = Pr_ab * P_o5

    # Nozzle exit, station e
    P_e = P_a
    T_e = T_o6 * (1 - eta_n * (1 - np.power((P_e / P_o6), ((gamma_e - 1) / gamma_e))))
    C_pe = (gamma_e * R) / (gamma_e - 1)
    u_e = np.sqrt(2 * C_pe * (T_o6 - T_e))
    ST = (1 + f + f_ab) * u_e - u
    TSFC = (f + f_ab) / ST
    #eta_th = (((1 + f + f_ab) * np.power(u_e, 2)) - np.power(u, 2)) / ((f + f_ab) * deltahr)
    #eta_o = u / (TSFC * deltahr)
    #eta_p = eta_o / eta_th
    return T_o4, T_o6, ST, TSFC

In [85]:
# Create the solution space for Pr_c, f, and f_ab
# got Pr_c range from https://en.wikipedia.org/wiki/Overall_pressure_ratio#Examples (did 10 to 60 for range of Pr_c (subsonic))
# got fuel-air ratio ranges from https://en.wikipedia.org/wiki/Jet_engine (did 0.001 to 0.021 for range of f and f_ab)
# note: had to decrease Pr_c range to 0 to 50 for supersonic
Pr_c = np.linspace(0, 50, 101, endpoint=True)
f = np.linspace(0.001, 0.021, 101, endpoint=True)
f_ab = np.linspace(0.001, 0.021, 101, endpoint=True)

# cols 0-2 : Pr_c, f, f_ab 
# cols 3-6 : T_o4, T_o6, ST, TSFC
o_tj = np.zeros((int(np.power(101,3)), 7))

for i in range(101):
    for j in range(101):
        for k in range(101):
            ind = i*10201 + j*101 + k
            o_tj[ind, 3], o_tj[ind, 4], o_tj[ind, 5], o_tj[ind, 6] = turbojet_fun(Pr_c[i], f[j], f_ab[k])
            o_tj[ind, 0], o_tj[ind, 1], o_tj[ind, 2] = Pr_c[i], f[j], f_ab[k]

print("Finished creating potential solution space.")

C:\Users\dell\AppData\Local\Temp\ipykernel_10840\3971656331.py:60: RuntimeWarning: divide by zero encountered in double_scalars
  T_o4 =  (((f * eta_b * deltahr) / (C_p4 * T_o3)) + 1) / ((f + 1) / T_o3)
C:\Users\dell\AppData\Local\Temp\ipykernel_10840\3971656331.py:60: RuntimeWarning: invalid value encountered in double_scalars
  T_o4 =  (((f * eta_b * deltahr) / (C_p4 * T_o3)) + 1) / ((f + 1) / T_o3)


lol u finished chuckin fr


In [86]:
# save to csv (just in case shenanigans happens)
jetpro_df = pd.DataFrame(o_tj)
jetpro_df.to_csv("output_turbojet_supersonic.csv")

In [87]:
# masked constraining based on given operating constraints
o_tj_copy = o_tj
ST_req = 750
row_to_delete = []
for row in range(len(o_tj[:,0])):
    if (o_tj[row, 3] > 1300 or o_tj[row, 4] > 2200 or o_tj[row, 5] < ST_req or np.isnan(np.sum(o_tj[row,:]))):
        row_to_delete.append(row)
o_tj_copy = np.delete(o_tj_copy, row_to_delete, axis=0)

In [89]:
# save masked array to csv (just in case shenanigans happens)
jetpro_df = pd.DataFrame(o_tj_copy)
jetpro_df.to_csv("output_turbojet_supersonic_masked.csv")

In [90]:
# find Pr_c, f, f_ab that produces minimum TSFC
min_TSFC_idx = np.argmin(o_tj_copy[:, 6])
min_TSFC = o_tj_copy[min_TSFC_idx, 6]
ST_ideal = o_tj_copy[min_TSFC_idx, 5]
Pr_c_ideal = o_tj_copy[min_TSFC_idx, 0]
f_ideal = o_tj_copy[min_TSFC_idx, 1]
f_ab_ideal = o_tj_copy[min_TSFC_idx, 2]

# create txt file of results
with open('turbojet_supersonic_res.txt', 'w') as f:
    f.write("idx = " + str(min_TSFC_idx) + "\n")
    f.write("minimum TSFC = " + str(min_TSFC) + "\n")
    f.write("ideal ST = " + str(ST_ideal) + "\n")
    f.write("ideal Pr_c = " + str(Pr_c_ideal) + "\n")
    f.write("ideal f = " + str(f_ideal) + "\n")
    f.write("ideal f_ab = " + str(f_ab_ideal))

print("idx = " + str(min_TSFC_idx))
print("minimum TSFC = " + str(min_TSFC))
print("ideal ST = " + str(ST_ideal))
print("ideal Pr_c = " + str(Pr_c_ideal))
print("ideal f = " + str(f_ideal))
print("ideal f_ab = " + str(f_ab_ideal))

idx = 0
minimum TSFC = 4.9329079975899494e-05
ideal ST = 750.0646681040259
ideal Pr_c = 1.5
ideal f = 0.02
ideal f_ab = 0.017
